In [1]:
from sys import path

path +=['..']

In [2]:
from mylib import Puller, PlotDisplayer, Slider, Time
import time
import numpy as np
import pandas as pd
import datetime
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot, layout
from jupyter_ui_poll import ui_events
import ipywidgets
import matplotlib
import scipy
from scipy import stats
import random
import os

output_notebook()
pl = Puller(simulate=False)  #simulate=True
random.seed()

# инициализация запускать только один раз

Loading BokehJS ...

[(13, 90113195), (13, 90113196), (13, 90113197)]


In [3]:
Time.SetZeroTime()
Time.unfreez()

In [4]:
pl.ms.StopButton() # кнопки для остоновки моторчиков в нештатной ситуации

GridspecLayout(children=(Button(description='Stop All Motors', layout=Layout(grid_area='widget001'), style=But…

In [5]:
pl.ms.motorM.Set_velocity(v=20, a=20)

0

In [6]:
pl.ms.MoveToStart(zapas=10) # сдвиг моторчиков на начальные позиции для растяжки
pl.ms.motorM.MoveTo(0);
while pl.ms.IsInMotion():
    pass

In [7]:
print(pl.tg.SetZeroWeight(0.5)) # калибровка тензодатчика
print(pl.tg.SetZeroWeight(0.5))
print(pl.tg.SetZeroWeight(3))

-1.0702406661267096
-0.1251786278438215
-0.031184851147057317


In [8]:
# pl.sim.l0=pl.ms.Distance(useL0=False)+2

In [12]:
pl.tg.ReadValue(tau=0)

0.020789900764704994

In [13]:
pl.SetW(5*2, dw=0.1, tau=1,quiet=False); # натяжка волокна

w=  13.645802850847847 , dw=  3.645802850847847 , dx=   -0.008257429468736602
w=  24.421203431606678 , dw=  14.421203431606678 , dx=   -0.032662783771509024
w=  11.379005685215144 , dw=  1.3790056852151444 , dx=   -0.0031233291125445078
w=  8.794128023470169 , dw=  -1.2058719765298314 , dx=   0.002731196173212011
w=  9.882132830156392 , dw=  -0.11786716984360801 , dx=   0.00026695898859063386
w=  10.124681672411283 , dw=  0.12468167241128292 , dx=   -0.00028239325001922817
w=  10.332580680058332 , dw=  0.3325806800583315 , dx=   -0.0007532665973991082
w=  10.353370580823038 , dw=  0.3533705808230376 , dx=   -0.000800353932137099
w=  9.660373888666205 , dw=  -0.3396261113337946 , dx=   0.0007692238924625091
w=  9.847482995548551 , dw=  -0.15251700445144856 , dx=   0.00034543787982061185
w=  10.062311970117168 , dw=  0.06231197011716816 , dx=   -0.00014113124580526374
SetW finish value= 10.17319144086226


In [14]:
x0=80
pl.ms.x0=x0
# pl.sim.hotFL=True

In [15]:
while pl.ms.IsInMotion():
    pass
dp=PlotDisplayer()
# dp2=PlotDisplayer()
# dp2.CreateMaket(mainParam= 'time',obr_coff=pd.DataFrame({
#     'time': pl.times,
#     'kof': pl.trueKmas
# }))
dp.CreateMaket(mainParam='time',power=pl.data[['time','power']],
                    tension=pl.data[['time','tension','tensionWgl', 'tensionEXPgl']],
                  motors=pl.data[['time','motorR','motorL','motorM']]
                #   v_and_a=pl.data[['time','vL','vR','aL','aR']]
         )
sl=Slider()
sl.NewSl('v',min=0.1,max=9,value=5,step=0.1)
sl.NewSl('a',min=0.1,max=9,value=5,step=0.1)
sl.NewSl('dhKof',min=0, max=3,value=1,step=0.1)
sl.NewSl('window',min=50, max=10000,value=100, step=50)
sl.NewSl('molibdenH',min=-10, max=2.5,value=0.0,step=0.1)
sl.NewSl('Kp',min=0, max=10,value=0.15,step=0.01)
sl.NewSl('Ki',min=0, max=10,value=0.4,step=0.01)
sl.NewSl('Kd',min=-100, max=100,value=0,step=0.01)
sl.NewSl('T0',min=0, max=200,value=5*2,step=0.01)
sl.NewBtn('molibden','up molibden','doun molibden')
sl.NewBtn('end','end','end2','end3','vse','bolshe ne jmi','nu pravda ne nado', 'samiy umniy da?')
sl.NewTt('x')
sl.NewTt('L')
sl.NewTt('R')
sl.NewTt('dv')
sl.NewTt('T')
sl.NewTt('ki')

In [16]:
pl.MotorsControlStart()
t0 = time.time()
t = t0
k = 1
ki = 0
dp.Show()
# dp2.Show()
sl.Display()
pl.Clear()
x=0

with ui_events() as poll:
    while True:
        poll(10)
        t = time.time()  
        T = sl.Sl['T0']() * (1 + 4 * x / pl.ms.xMax) * k**2
        pl.Tgoal = T
        if pl.PulMotorsControl(sl.Sl['molibdenH'](), T,
                                    upFl=sl.BtnFl['molibden'], stFl=sl.BtnFl['end'],
                                    dhKof=sl.Sl['dhKof'](), Kp=sl.Sl['Kp']()*0.001,Ki=ki,Kd=sl.Sl['Kd']()
                                 )==-1:
            break
        x=pl.data['x'].iloc[-1]
        pl.v=sl.Sl['v']()
        pl.a=sl.Sl['a']()
        ww=int(sl.Sl['window']())
        k = pl.ms.R_x(x)/pl.ms.R_x(0)
        ki = sl.Sl['Ki']()
        sl.ChangeValueTt('x',x)
        sl.ChangeValueTt('L',pl.ms.L_x(x))
        sl.ChangeValueTt('R',pl.ms.R_x(x))
        sl.ChangeValueTt('dv',pl.dv)
        sl.ChangeValueTt('T',T)
        sl.ChangeValueTt('ki',ki)
        dp.Apdate(for_all=pl.data.iloc[-ww:])
#         dp2.Apdate(for_all=pd.DataFrame({
#                     'time': pl.times,
#                     'kof': pl.trueKmas
#                 }))
        push_notebook()    
        Time.sleep(0.04)
pl.ms.motorM.MoveTo(0);
print('ura')
pl.Save()

# with ui_events() as poll:
#     while sl.BtnFl['end']<3:
#         poll(10)
#         pl.SetW(2,dw=100,tau=0.5)
print('vse')

слишком поздний вызов PulFireMove
xMax= 34.46077459566587 L_x(xMax)= 12.0   x= 33.238990628426315   L= 15.816571166114944 xEnd= 35.366149342578034 lEnd= 12.0
ura


INFO - poolling - Running command 'push_to_mango'
INFO - poolling - Started run with ID "749"
INFO - poolling - Completed after 0:00:07


Added text/csv as content-type of artifact C:\Users\ariva\projects\table_control\data\pull_resalts.csv.
vse


In [30]:
pl.ms.motorL.MoveTo(90)
pl.ms.motorR.MoveTo(47)

0

In [24]:
pl.ms.motorL.Getposition(), pl.ms.motorR.Getposition()

(71.20955925765719, 71.20956153091703)

In [18]:
pl.ms.End() # растановка моторчиков на нули (запускать перед отключением питания)

ending
end


In [27]:
del pl

In [26]:
pl.ms.Move(60, v=1, a=1, vdiff=0, da=0)

error in  motor L  -  position is bad x= -60.0 min position= 0


-1

In [30]:
pl.SetW(5, dw=0.1, tau=1,quiet=False);

w=  15.979657434853443 , dw=  10.979657434853443 , dx=   -0.029161678727538005
w=  13.746084386567972 , dw=  8.746084386567972 , dx=   -0.02322936799425163
w=  7.751662999411395 , dw=  2.751662999411395 , dx=   -0.007308343892457678
w=  5.880571930587955 , dw=  0.8805719305879549 , dx=   -0.002338775675712746
w=  5.243014973803672 , dw=  0.24301497380367199 , dx=   -0.0006454413203774368
w=  5.152925403823284 , dw=  0.1529254038232839 , dx=   -0.0004061658136452728
w=  5.055905866921328 , dw=  0.05590586692132771 , dx=   -0.00014848449870294284
SetW finish value= 5.090555701529169


In [37]:
pl.tg.ReadValue(tau=0)

5.340034510705627

In [22]:
pl.ms.motorM.MoveTo(78)

0

In [23]:
pl.ms.motorM.MoveTo(0)

0

### Для калибровки

In [2]:
pl.Clear()

dp1 = PlotDisplayer()
dp1.CreateMaket(mainParam='time', tension=pl.data[['time', 'tension']])

sl1 = Slider()
sl1.NewBtn('stop', 'stop', 'stopped')
sl1.NewTt('Tension')

dp1.Show()
sl1.Display()

with ui_events() as poll:
    while not sl1.BtnFl['stop']:
        poll(10)
        
        pl.Read(False, False, False)
        
        dp1.Apdate(for_all=pl.data.iloc[-200:])
        sl1.ChangeValueTt('Tension', pl.tg.ReadValue(tau=0))
        Time.sleep(.04)
        push_notebook()

NameError: name 'pl' is not defined

In [47]:
pl.tg.weightCoef = -0.0075585384235655265

In [50]:
pl.tg.weightCoef *= 265598 / 1000 ** 2

In [37]:
pl.tg.weightCoef *= 1000 / 9.8

In [54]:
-0.0075585384235655265 * 265598 / 1000 ** 2

-0.0020075326882221566